# Physics Tutorial 01: Entropy Visualized

Interactive exploration of entropy, microstates, and the Second Law.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb
from math import factorial, log

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Boltzmann constant (we'll use k_B = 1 for simplicity in most examples)
k_B = 1.38e-23  # J/K

## Part 1: Microstates and Macrostates

Let's start with the simplest example: coins.

In [ ]:
def count_microstates_coins(n_coins, n_heads):
    """Number of ways to get n_heads from n_coins."""
    return comb(n_coins, n_heads, exact=True)

def entropy_boltzmann(W):
    """S = k_B ln(W), using k_B = 1 for simplicity."""
    if W <= 0:
        return 0
    return np.log(W)

# Analyze for different numbers of coins
for n_coins in [4, 10, 20]:
    print(f"\n{'='*60}")
    print(f"N = {n_coins} coins")
    print(f"{'='*60}")
    print(f"{'Heads':>6} | {'Microstates (W)':>15} | {'S = ln(W)':>10} | {'Probability':>12}")
    print(f"{'-'*60}")
    
    total_microstates = 2**n_coins
    
    for n_heads in range(n_coins + 1):
        W = count_microstates_coins(n_coins, n_heads)
        S = entropy_boltzmann(W)
        prob = W / total_microstates
        print(f"{n_heads:>6} | {W:>15,} | {S:>10.4f} | {prob:>12.4%}")
    
    print(f"{'-'*60}")
    print(f"{'Total':>6} | {total_microstates:>15,} |            | {'100%':>12}")

In [ ]:
# Visualize entropy vs macrostate
n_coins = 100

heads = np.arange(0, n_coins + 1)
microstates = [count_microstates_coins(n_coins, h) for h in heads]
entropies = [entropy_boltzmann(W) for W in microstates]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Microstates
axes[0].plot(heads, microstates, 'b-', linewidth=2)
axes[0].fill_between(heads, microstates, alpha=0.3)
axes[0].set_xlabel('Number of Heads', fontsize=12)
axes[0].set_ylabel('Number of Microstates W', fontsize=12)
axes[0].set_title(f'Microstates vs Macrostate (N={n_coins} coins)', fontsize=14)
axes[0].axvline(x=n_coins/2, color='red', linestyle='--', label=f'Max at {n_coins//2} heads')
axes[0].legend(fontsize=11)

# Entropy
axes[1].plot(heads, entropies, 'g-', linewidth=2)
axes[1].fill_between(heads, entropies, alpha=0.3, color='green')
axes[1].set_xlabel('Number of Heads', fontsize=12)
axes[1].set_ylabel('Entropy S = ln(W)', fontsize=12)
axes[1].set_title(f'Entropy vs Macrostate (N={n_coins} coins)', fontsize=14)
axes[1].axvline(x=n_coins/2, color='red', linestyle='--', label='Maximum entropy')
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

# Show maximum
max_idx = np.argmax(entropies)
print(f"\nMaximum entropy at {heads[max_idx]} heads")
print(f"S_max = ln({microstates[max_idx]:,}) = {entropies[max_idx]:.4f}")
print(f"Maximum possible S = ln(2^{n_coins}) = {n_coins * np.log(2):.4f}")

## Part 2: Gas Molecules in a Box

The classic entropy example: molecules spreading out.

In [ ]:
def simulate_gas_expansion(n_molecules, n_steps):
    """
    Simulate gas molecules in a 2D box.
    Start: all on left half.
    Each step: random molecule moves to random new position.
    """
    # Positions: True = left, False = right
    positions = np.ones(n_molecules, dtype=bool)  # All start on left
    
    history = {'step': [0], 'n_left': [n_molecules], 'entropy': [0]}  # S=0 for 1 microstate
    
    for step in range(1, n_steps + 1):
        # Pick a random molecule, move to random half
        mol_idx = np.random.randint(n_molecules)
        positions[mol_idx] = np.random.random() < 0.5
        
        n_left = positions.sum()
        W = comb(n_molecules, n_left, exact=True)
        S = np.log(W) if W > 0 else 0
        
        history['step'].append(step)
        history['n_left'].append(n_left)
        history['entropy'].append(S)
    
    return history

# Run simulation
np.random.seed(42)
n_molecules = 100
n_steps = 500

history = simulate_gas_expansion(n_molecules, n_steps)

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Number on left side
axes[0].plot(history['step'], history['n_left'], 'b-', linewidth=1, alpha=0.7)
axes[0].axhline(y=n_molecules/2, color='red', linestyle='--', linewidth=2, label='Equilibrium')
axes[0].set_ylabel('Molecules on Left Side', fontsize=12)
axes[0].set_title(f'Gas Expansion: {n_molecules} Molecules', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].set_ylim(0, n_molecules)

# Entropy
max_entropy = n_molecules * np.log(2)
axes[1].plot(history['step'], history['entropy'], 'g-', linewidth=1, alpha=0.7)
axes[1].axhline(y=max_entropy, color='red', linestyle='--', linewidth=2, label=f'Max S = N ln(2) = {max_entropy:.1f}')
axes[1].set_xlabel('Time Steps', fontsize=12)
axes[1].set_ylabel('Entropy S = ln(W)', fontsize=12)
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

print(f"\nStarting entropy: S = ln(1) = 0 (all on one side)")
print(f"Equilibrium entropy: S = ln(C({n_molecules},{n_molecules//2})) ≈ {np.log(comb(n_molecules, n_molecules//2, exact=True)):.2f}")
print(f"Maximum possible: S = {n_molecules} × ln(2) = {max_entropy:.2f}")

## Part 3: Why Entropy Increases — Probability

Let's see why high entropy states are more probable.

In [ ]:
# Probability of different macrostates
n_molecules = 20

n_left_vals = np.arange(0, n_molecules + 1)
microstates = [comb(n_molecules, n, exact=True) for n in n_left_vals]
total = sum(microstates)
probabilities = [W / total for W in microstates]

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(n_left_vals, probabilities, color='steelblue', edgecolor='black', alpha=0.7)

# Color extreme states
bars[0].set_color('red')
bars[-1].set_color('red')

ax.set_xlabel('Number of Molecules on Left', fontsize=12)
ax.set_ylabel('Probability', fontsize=12)
ax.set_title(f'Probability of Each Macrostate (N={n_molecules} molecules)', fontsize=14)

# Annotate extremes
ax.annotate(f'All left\nP = {probabilities[0]:.2e}', 
            xy=(0, probabilities[0]), xytext=(2, max(probabilities)/2),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=11, color='red')

ax.annotate(f'Half-half\nP = {probabilities[n_molecules//2]:.2%}', 
            xy=(n_molecules//2, probabilities[n_molecules//2]), 
            xytext=(n_molecules//2 + 3, probabilities[n_molecules//2] + 0.05),
            arrowprops=dict(arrowstyle='->', color='green'),
            fontsize=11, color='green')

plt.tight_layout()
plt.show()

print(f"\nProbability comparisons:")
print(f"  All on left: {probabilities[0]:.2e}")
print(f"  Half and half: {probabilities[n_molecules//2]:.4f}")
print(f"  Ratio: {probabilities[n_molecules//2] / probabilities[0]:,.0f}x more likely!")

In [ ]:
# Show how this scales with N
n_values = [10, 20, 50, 100, 200]

print("Probability that ALL molecules stay on one side:")
print("=" * 50)
print(f"{'N':>5} | {'P(all left)':>20} | {'1 in ...':<20}")
print("-" * 50)

for n in n_values:
    p = 1 / (2**n)
    print(f"{n:>5} | {p:>20.2e} | {2**n:>20,}")

print("\n→ For N = Avogadro's number (6×10²³), this is 1 in 10^(10²³)!")
print("   That's a number with 100,000,000,000,000,000,000,000 digits!")

## Part 4: Gibbs Entropy for Non-Equal Probabilities

In [ ]:
def gibbs_entropy(probs):
    """S = -Σ p_i ln(p_i)"""
    probs = np.array(probs)
    # Avoid log(0)
    mask = probs > 0
    return -np.sum(probs[mask] * np.log(probs[mask]))

# Compare different probability distributions (3 states)
distributions = [
    ([1.0, 0.0, 0.0], "Certain (one state)"),
    ([0.5, 0.5, 0.0], "Two states, equal"),
    ([0.7, 0.2, 0.1], "Three states, unequal"),
    ([1/3, 1/3, 1/3], "Three states, equal (max)")
]

print("Gibbs Entropy for Different Distributions:")
print("=" * 60)

for probs, name in distributions:
    S = gibbs_entropy(probs)
    print(f"  {name}")
    print(f"    p = {probs}")
    print(f"    S = {S:.4f}")
    print()

In [ ]:
# Visualize: entropy is maximized for uniform distribution
# For 2 states: vary p from 0 to 1

p_vals = np.linspace(0.001, 0.999, 100)
entropies = [-p*np.log(p) - (1-p)*np.log(1-p) for p in p_vals]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(p_vals, entropies, 'b-', linewidth=2)
ax.fill_between(p_vals, entropies, alpha=0.3)
ax.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Maximum at p=0.5')
ax.plot(0.5, np.log(2), 'ro', markersize=10)

ax.set_xlabel('Probability p of state 1', fontsize=12)
ax.set_ylabel('Entropy S = -p ln(p) - (1-p) ln(1-p)', fontsize=12)
ax.set_title('Gibbs Entropy for 2-State System', fontsize=14)
ax.legend(fontsize=11)

ax.annotate(f'S_max = ln(2) = {np.log(2):.3f}', xy=(0.5, np.log(2)), 
            xytext=(0.65, np.log(2)-0.1), fontsize=12,
            arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

print("\nKey insight: Entropy is maximized when probabilities are EQUAL.")
print("This is why systems tend toward 'maximum ignorance' - all states equally likely.")

## Part 5: Heat Flow and Entropy

In [ ]:
def entropy_change_heat_transfer(Q, T_hot, T_cold):
    """
    Calculate entropy change when heat Q flows from hot to cold.
    
    dS_hot = -Q/T_hot (loses heat)
    dS_cold = +Q/T_cold (gains heat)
    dS_total = Q(1/T_cold - 1/T_hot) > 0 when T_hot > T_cold
    """
    dS_hot = -Q / T_hot
    dS_cold = Q / T_cold
    dS_total = dS_hot + dS_cold
    return dS_hot, dS_cold, dS_total

# Example: heat flow between two objects
T_hot = 400  # K
T_cold = 300  # K
Q = 100  # J

dS_hot, dS_cold, dS_total = entropy_change_heat_transfer(Q, T_hot, T_cold)

print("Heat Transfer Entropy Example:")
print("=" * 50)
print(f"Hot object: T = {T_hot} K")
print(f"Cold object: T = {T_cold} K")
print(f"Heat transferred: Q = {Q} J")
print(f"\nEntropy changes:")
print(f"  Hot object: ΔS = {dS_hot:.4f} J/K (decreases)")
print(f"  Cold object: ΔS = {dS_cold:.4f} J/K (increases)")
print(f"  Total: ΔS = {dS_total:.4f} J/K (net INCREASE)")
print(f"\n>>> Second Law satisfied: ΔS_total > 0 <<<")

In [ ]:
# Visualize entropy change vs temperature difference
T_hot = 400
T_cold_vals = np.linspace(100, 399, 100)
Q = 100

dS_totals = [Q * (1/T_cold - 1/T_hot) for T_cold in T_cold_vals]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(T_cold_vals, dS_totals, 'g-', linewidth=2)
ax.fill_between(T_cold_vals, dS_totals, alpha=0.3, color='green')
ax.axhline(y=0, color='black', linewidth=0.5)

ax.set_xlabel('Cold Temperature (K)', fontsize=12)
ax.set_ylabel('Total Entropy Change ΔS (J/K)', fontsize=12)
ax.set_title(f'Entropy Change vs Temperature Difference (T_hot = {T_hot}K, Q = {Q}J)', fontsize=14)

ax.annotate('Larger T difference\n= Larger ΔS', xy=(200, Q*(1/200-1/T_hot)), 
            xytext=(220, 0.3), fontsize=11,
            arrowprops=dict(arrowstyle='->', color='green'))

ax.annotate('As T_cold → T_hot\nΔS → 0', xy=(380, Q*(1/380-1/T_hot)), 
            xytext=(320, 0.1), fontsize=11,
            arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

print("\nKey insight: Heat flow from hot to cold ALWAYS increases total entropy.")
print("The larger the temperature difference, the larger the entropy increase.")

## Summary

| Concept | Formula | Meaning |
|---------|---------|--------|
| Boltzmann Entropy | $S = k_B \ln W$ | More microstates = higher entropy |
| Gibbs Entropy | $S = -k_B \sum p_i \ln p_i$ | General case with probabilities |
| Second Law | $\Delta S_{total} \geq 0$ | Entropy never decreases (isolated system) |
| Heat Transfer | $\Delta S = Q/T_{cold} - Q/T_{hot}$ | Heat flow increases entropy |

**The deep insight**: Entropy increase is NOT a mysterious force—it's just probability. Systems evolve toward states with more microstates because those states are more probable.